In [3]:
# Data file at https://www.cse.ust.hk/msbd5003/data

lines = sc.textFile('/Users/zhaocai/adj_noun_pairs.txt', 8)

In [4]:
lines.count()

3162692

In [5]:
lines.getNumPartitions()

8

In [6]:
lines.take(5)

['early radical',
 'french revolution',
 'pejorative way',
 'violent means',
 'positive label']

In [7]:
# Converting lines into word pairs. 
# Data is dirty: some lines have more than 2 words, so filter them out.
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()

PythonRDD[4] at RDD at PythonRDD.scala:49

In [8]:
pairs.take(5)

[('early', 'radical'),
 ('french', 'revolution'),
 ('pejorative', 'way'),
 ('violent', 'means'),
 ('positive', 'label')]

In [9]:
N = pairs.count()

In [10]:
N

3162674

In [11]:
# Compute the frequency of each pair.
# Ignore pairs that not frequent enough
pair_freqs = pairs.map(lambda p: (p,1)).reduceByKey(lambda f1, f2: f1 + f2) \
                  .filter(lambda pf: pf[1] >= 100)

In [12]:
pair_freqs.take(5)
pair_freqs.

[(('16th', 'century'), 950),
 (('civil', 'war'), 2236),
 (('social', 'class'), 155),
 (('sixteenth', 'century'), 137),
 (('late', '1970'), 444)]

In [13]:
# Computing the frequencies of the adjectives and the nouns
a_freqs = pairs.map(lambda p: (p[0],1)).reduceByKey(lambda x,y: x+y)
n_freqs = pairs.map(lambda p: (p[1],1)).reduceByKey(lambda x,y: x+y)

In [14]:
a_freqs.take(5)

[('social', 7413),
 ('free', 6635),
 ('stoic', 72),
 ('good', 7499),
 ('other', 75260)]

In [15]:
n_freqs.count()

106333

In [72]:
# Broadcasting the adjective and noun frequencies. 
#a_dict = a_freqs.collectAsMap()
#a_dict = sc.parallelize(a_dict).map(lambda x: x)
#n_dict = sc.broadcast(n_freqs.collectAsMap())
#a_dict = sc.broadcast(a_freqs.collectAsMap())

   
    
    

TypeError: 'builtin_function_or_method' object is not iterable

In [54]:
# Computing the PMI for all pairs.
scored_pairs = pair_freqs.map(pmi_score(n_dict,a_dict))

TypeError: pmi_score() missing 1 required positional argument: 'a_dict'

In [49]:
# Printing the most strongly associated pairs. 
scored_pairs.top(10)
#print(scored_pairs)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 63.0 failed 1 times, most recent failure: Lost task 1.0 in stage 63.0 (TID 252, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 230, in main
    process()
  File "/usr/local/Cellar/apache-spark/2.3.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 225, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/2.3.1/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/Cellar/apache-spark/2.3.1/libexec/python/pyspark/rdd.py", line 857, in func
    initial = next(iterator)
  File "/usr/

In [74]:
products = {1: "Apple", 2: "Orange", 3: "TV", 5: "Computer"}
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

# broadcasted_products = sc.broadcast(products)

def f(x):
    return (x[0], broadcasted_products.value[x[0]], x[1])

# results = trans.map(lambda x: (x[0], broadcasted_products.value[x[0]], x[1]))
results = trans.map(lambda x: (x[0], products[x[0]], x[1]))
print (results.collect())

[(1, 'Apple', (134, 'OK')), (3, 'TV', (34, 'OK')), (5, 'Computer', (162, 'Error')), (1, 'Apple', (135, 'OK')), (2, 'Orange', (53, 'OK')), (1, 'Apple', (45, 'OK'))]
